In [2]:
import scanpy as sc
import pandas as pd
import numpy as np

In [ ]:
adata.layers['raw']=adata.raw.X
adatar1=adata[adata.obs['replicate']=='1']

In [1]:
import json
ob=open('/media/sergio/xenium_b_and_heart/Xenium_benchmarking-main/figures/polygons/regions_multisection_1.json')
js=json.load(ob)
output=pd.DataFrame(columns=['y','x','region_annotation','region_general'])
nu=0
for ob in js:
    for num in range(0,len(ob['coordinates'][0])):
        output.loc[nu,:]=list([ob['coordinates'][0][num][0],ob['coordinates'][0][num][1],ob['name'],ob['name'].replace(' [1]','').replace('D: ','')])
        nu=nu+1

from shapely.geometry import Point, Polygon
def domainassign(plsin,adatadom):
    adatadom.obs['region_annotation']='None'
    plt.figure()
    for sel in plsin['region_annotation'].unique():
        plsub=plsin[plsin['region_annotation']==sel]
        if plsub.shape[0]>2:
    #        plt.figure()
            coord = np.array(plsub[['y','x']]).tolist()
            coord.append(coord[0])
            poli=Polygon(coord)
            xs, ys = zip(*coord) #create lists of x and y values
            plt.plot(xs,ys,color='black') 
            plt.title(sel)
            for n in adatadom.obs.index:
                pnt=Point(adatadom.obs.loc[n,'y_centroid'],adatadom.obs.loc[n,'x_centroid'])
                if pnt.within(poli)==True:
                    adatadom.obs.loc[n,'region_annotation']=sel
    plt.scatter(adatadom.obs['y_centroid'],adatadom.obs['x_centroid'],s=0.5)
    plt.show() # if you need...
    return adatadom

dictio_region=dict(zip(output['region_annotation'],output['region_general']))
dictio_region['D: vl [2]']='vl'

FileNotFoundError: [Errno 2] No such file or directory: '/media/sergio/xenium_b_and_heart/Xenium_benchmarking-main/figures/polygons/regions_multisection_1.json'

In [ ]:
nancells=adatar1[adatar1.obs['region_annotation']=='None']
annotatedcells=adatar1[adatar1.obs['region_annotation']!='None']
from tqdm import tqdm
nancells.obs['new_domain']='nan'
print('caclulating distance to vessel...')
for rd in tqdm(nancells.obs.index):
        meas=((annotatedcells.obs.X-nancells.obs.loc[rd,'X'])**2)+((annotatedcells.obs.Y-nancells.obs.loc[rd,'Y'])**2)
        nancells.obs.loc[rd,'new_domain']=meas.idxmin()
dican=dict(zip(annotatedcells.obs.index,annotatedcells.obs['region_annotation']))
nancells.obs['new_domain']=nancells.obs['new_domain'].map(dican)
new_assignment=dict(zip(nancells.obs.index,nancells.obs['new_domain']))
for indi in nancells.obs.index:
    adatar1.obs.loc[indi,'region_annotation']=new_assignment[indi]